<a href="https://colab.research.google.com/github/AilingLiu/Growth_Analysis/blob/master/Monthly_trending_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
url='https://github.com/AilingLiu/Growth_Analysis/blob/master/Data/online_retail.csv?raw=true'
retail = pd.read_csv(url, encoding = 'unicode_escape')
retail['InvoiceDate']=pd.to_datetime(retail['InvoiceDate'])
retail['Payment'] = retail['Quantity'] * retail['UnitPrice']
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Payment
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [0]:
retail['InvoiceYM'] = retail['InvoiceDate'].apply(lambda x: x.strftime('%Y%m'))
df = retail.loc[(retail['Quantity']>0) &(retail['InvoiceYM']!='201112')] #filter out the negative(canceled) quantity

In [0]:
trending = df.groupby(['InvoiceYM', 'StockCode']).agg({'Quantity': {'UnitsSold': 'sum'},
                                                'Payment': {'TotalSale':'sum'}}).droplevel(level=0, axis=1).reset_index() # we sort by first quantity, then tota sale if quantity is equal

topFive = trending.groupby(['InvoiceYM']).apply(lambda sf: sf.nlargest(5, columns=['UnitsSold', 'TotalSale'])).drop(columns=['InvoiceYM'])

Here our trendy products are defined as the most bought items, regardless of its price. In reality, we should segment the most bought items from low to high price (or net revenue) category, so marketing team can design price strategy according to different product category. 

In [68]:
topFive = topFive.droplevel(level=1, axis=0)
topFive.loc['201103'] #example of top 5 in 2011 March

,StockCode,UnitsSold,TotalSale
InvoiceYM,,,
201103,85099B,5282,10118.38
201103,21212,3975,2293.53
201103,84077,3888,943.20
201103,22616,3645,1014.69
201103,84988,3635,925.74


In [69]:
top5Prod = topFive.loc['201012':'201111'].StockCode.unique() #the the top 5 products from December 2010 to November 2011.
top5Prod

array(['84077', '21212', '85123A', '22834', '22197', '23166', '37413',
       '21108', '22053', '22693', '85099B', '22616', '84988', '22440',
       '15036', '21977', '17003', '84568', '18007', '23343', '84879',
       '23203', '23199', '22952', '23084', '20971', '84826', '22086'],
      dtype=object)

Let's make an interactive plot where users can choose one of the listed top5 products as above and see its sales among the year. 
Note: Interactive plot is not supported in GitHub. Open in Colab or download in your jupyter notebook for view.

In [95]:
def UnitsSold(code, data=df):
  return data.loc[data['StockCode']==code].groupby(['InvoiceYM'])\
  .agg({'Quantity': {'UnitsSold': 'sum'}, 'Payment': {'TotalSale': 'sum'}})\
  .droplevel(level=0, axis=1)

def VizProdSales(code, data=df):

  table = UnitsSold(code)
  
  ax1 = table.loc[:, 'UnitsSold'].plot(kind='line', 
                                      grid=True, 
                                      figsize=(15, 8), 
                                      title='Sales of Product ' + code, 
                                      rot=45)

  ax2 = table.loc[:, 'TotalSale'].plot(kind='bar', 
                                      alpha=0.2, 
                                      color='teal', 
                                      secondary_y=True, 
                                      rot=45)

  ax1.legend(['UnitsSales'], loc='upper left')
  ax1.set_ylabel('Units', fontdict={'fontsize': 12, 'fontweight': 'medium'})
  ax1.set_ylim(0, table.loc[:, 'UnitsSold'].max()*1.2)

  ax2.legend(['SalesAmount'], loc='upper right')
  ax2.set_ylabel('Revenue($)', fontdict={'fontsize': 12, 'fontweight': 'medium'})
  ax2.set_ylim(0, table.loc[:, 'TotalSale'].max()*1.2)
  plt.show()

from ipywidgets import interact, fixed
interact(VizProdSales, code=top5Prod, data=fixed(df))

interactive(children=(Dropdown(description='code', options=('84077', '21212', '85123A', '22834', '22197', '231…

<function __main__.VizProdSales>

In [94]:
df.loc[(df.StockCode=='84826') & (df.InvoiceYM=='201111')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Payment,InvoiceYM
452377,575336,84826,ASSTD DESIGN 3D PAPER STICKERS,4,2011-11-09 13:58:00,0.85,14968.0,United Kingdom,3.40,201111
452421,575337,84826,ASSTD DESIGN 3D PAPER STICKERS,5,2011-11-09 14:11:00,0.85,17867.0,United Kingdom,4.25,201111
452454,575337,84826,ASSTD DESIGN 3D PAPER STICKERS,1,2011-11-09 14:11:00,0.85,17867.0,United Kingdom,0.85,201111
458281,575767,84826,ASSTD DESIGN 3D PAPER STICKERS,1,2011-11-11 11:11:00,0.85,17348.0,United Kingdom,0.85,201111
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.00,13256.0,United Kingdom,0.00,201111


From the product 84826, it was sold almost zero items in the past 11 months, until it suddenly spikes in November, for over 10,000 units. Strangely, the revenue was not high at all. Looking further back at the data, we found this item was sold 12540 units on 2011-11-25, but UnitPrice was changed to 0, thus it only boost the unit sold amount but not the revenue. It might be company internal clean out of this item.

Other top products demonstrate spikes in various months. Whether those fluctuation is seasonal effects or new business growth, we should further look back on previous years sales data to validate. 

The product trend is the key business driver. Marketing team can stimulate potential clients by recommending trending products, leading to higher conversion rate. Moreover, we should do the same to identify the monthly least bought products to improve sales.
